In [1]:
from pathlib import Path


def get_files():
    parent = Path('../data/')
    return [child.absolute() for child in parent.iterdir()]


1. 파일 불러오기

In [2]:
from array import array
import numpy as np
import plotly.express as px
import cv2


def load_ir_image(file_name):
    WIDTH = 160
    HEIGHT = 120

    with open(file_name, 'rb') as file:
        bytes = file.read(WIDTH * HEIGHT * 2)

    ir_array = array('H')
    ir_array.frombytes(bytes)

    ir_image = np.array(ir_array).reshape(HEIGHT, WIDTH)
    ir_image = (ir_image - 27315) / 100
    return ir_image


def show_image(img, title=""):
    fig = px.imshow(img, title=title)
    fig.show()


2. 히스토그램 출력

In [3]:
def show_histogram(img):
    fig = px.histogram(img.ravel())
    fig.show()

3. Threshold 이진화

In [4]:
def threshold(img, start, end):
    bin_img = np.zeros(shape=img.shape,
                       dtype=np.uint8)

    for row, col in np.ndindex(bin_img.shape):
        bin_img[row, col] = 0xFF if start <= img[row, col] and img[row, col] <= end else 0

    return bin_img


4. 영역 내 평균 온도 구하기

In [5]:
def get_average_tempeature(img, region):
    num_pixels = 0
    sum_temperature = 0
    for y, x in np.ndindex(img.shape):
        if region[y, x] != 0:
            sum_temperature += img[y, x]
            num_pixels += 1
    
    return sum_temperature / num_pixels

# main

In [6]:
img = load_ir_image('../data/irimage_20220716_0735.dat')

hist, bin_edges = np.histogram(img, bins=256)
FREQUENCY = 500
indices = np.argwhere(hist > FREQUENCY)
start = bin_edges[indices[0, 0]]
end = bin_edges[indices[-1, 0] + 1]
bin_img = threshold(img, start, end)

# 열화상 이미지 출력
show_image(img, '열화상 이미지')

# 필터 영역 표시
show_image(bin_img, f'히스토그램 내 빈도수가 500 이상인 영역({start:.2f}~{end:.2f})')

# 히스토그램 출력
show_histogram(img)
print(f"잎의 평균 온도: {get_average_tempeature(img, bin_img):.2f}")



잎의 평균 온도: 26.90
